In [ ]:
import pandas as pd
import numpy as np
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup

import json

from time import sleep

import scraper_walmart as walmart

# Our generated code
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%load_ext autoreload
%autoreload 2

In [ ]:
products = pd.read_pickle("./products_pd.pickle")

products['price_dollar'] = np.nan



View the products from Walmart stores

Request products details from Walmart Open API

In [ ]:
products.product_name = products.product_name.fillna("")
products.brands = products.brands.fillna("")

scraped_products = []

with open('walmart_products.json', mode='w', encoding='utf-8') as f:

    count = 0
    for i, row in products.iterrows():
        if "Walmart" in row.stores:
            count = count+1
            query = row['product_name'] + row["brands"]
            print(query.title())
            try:
                products_dicts = walmart.search(query)
            except Exception:
                continue

            for product_dict in products_dicts:
                item = {'code': row.name, 
                        'product_name': row.product_name,
                        'store_product_name': product_dict.get('name', ""),
                        'store_price': product_dict.get('salePrice', ""),
                        'store_currency': 'USD',
                        'store_category_path' : product_dict.get('categoryPath', "")
                        }
                scraped_products.append(item)
                
            # Be nice to te server
            timer.sleep(0.1)

    json.dump(scraped_products, f)

In [ ]:
with open('walmart_products.json', mode='w', encoding='utf-8') as f:
    json.dump(scraped_products, f)

# Cleansing

In [ ]:
from fuzzyset import FuzzySet

def compute_match_score(product_name_list, store_name_list):
    
    if product_name_list and store_name_list:
        name_fs = FuzzySet(store_name_list)
        name_score = [0 if name_fs.get(i)==None else name_fs.get(i)[0][0] for i in product_name_list]
        return sum(name_score) / len(name_score)  
    else:
        return 0

In [ ]:
# Import crawled data
with open('walmart_products.json', mode='r', encoding='utf-8') as f:
    data = json.load(f)
    
walmart_products = pd.DataFrame.from_dict(data)

walmart_products = walmart_products.drop_duplicates(subset='product_name')
walmart_products

In [ ]:
# Compute match score
walmart_products['match_score'] = walmart_products.apply(lambda row: compute_match_score(row.product_name.lower().split(),
                                                       row.store_product_name.lower().replace('-', ' ').split()), 
                       axis=1)
walmart_products